# One symbol program
## Run an entire cycle upto df_nakeds for one symbol
- [x] Get chains
- [x] Get a symbol
- [x] Update its chain strikes
- [x] Get its closest SD
- [ ] Get its margin
- [ ] Get its expPrice

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'.upper()
symbol = 'NIFTY50'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

In [2]:
# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
COLS = _vars.COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()


# One symbol chain

In [3]:
# Imports
from utils import get_pickle, make_chains, get_strike_closest_to_und, get_margins, make_a_raw_contract, qualify_naked_opts, get_exchange_name
from ib_insync import Option
import asyncio

In [4]:
# Get one symbol
UNDS_PATH = DATAPATH / 'unds.pkl'

# Get the underling contract for the symbol
contract = get_pickle(UNDS_PATH)[symbol]


In [5]:
# Get all the chains of the symbol
df_sym_chain = asyncio.run(make_chains(contracts=contract))

# Remove fractional strikes from df_sym_chain
only_whole = df_sym_chain.strike.apply(lambda x: x%1==0)
df_sym_chain = df_sym_chain[only_whole]

# Get the closest strikes to und
df_ch = get_strike_closest_to_und(df_sym_chain, 1)

Getting NSE chains: 100%|██████████| 1/1 [00:09<00:00,  9.73s/it]


In [6]:
# Get margins and commissions from the options closest to und
df_chq = qualify_naked_opts(df_ch)

NSE naked quals:   0%|          | 0/18 [00:00<?, ?it/s]

NSE naked quals:   6%|▌         | 1/18 [00:00<00:10,  1.68it/s]Error 200, reqId 43430: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='NIFTY50', lastTradeDateOrContractMonth='20240926', strike=22150.0, right='C', multiplier=1, exchange='NSE', currency='INR')
Error 200, reqId 43432: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='NIFTY50', lastTradeDateOrContractMonth='20250327', strike=22150.0, right='C', multiplier=1, exchange='NSE', currency='INR')
Error 200, reqId 43431: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='NIFTY50', lastTradeDateOrContractMonth='20241226', strike=22150.0, right='C', multiplier=1, exchange='NSE', currency='INR')
Error 200, reqId 43434: No security definition has been found for the request, contract: Contract(secType='OPT', symbol='NIFTY50', lastTradeDateOrContractMonth='20251224', strike=22150.0, right='C', mul

In [8]:
# REplace nan | Zero values of margin with mgnCompute
from utils import snp_margin_compute

if MARKET == 'SNP':
    df = snp_margin_compute(df_chq)
    bad_margin = df.margin == 0 | df.margin.isnull()
    df_chqm = df.loc[bad_margin, 'margin'] = df[bad_margin].mgnCompute
    

# NSE margin compute

In [ ]:
from utils import get_pickle
get_pickle(DATAPATH / 'df_und_prices.pkl')

In [ ]:
# Compute margins from get_unds_with_prices.
# TODO: Integrate `lots` and `multiplier` to output of `get_unds_with_prices`
# TODO: Modify get_unds_with_prices to put the lot and multiplier fields.

In [9]:
df_chq

,conId,symbol,dte,expiry,strike,multiplier,undPrice,iv,sigma,strike_sdev,right,market,lot,strk_near_und,contract,margin,comm
0,680592042,NIFTY50,6.035213,20240425,22150.0,1,22144.95,0.131166,373.505295,0.013521,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=680592042, symbo...",153542.47,20.0
1,692768220,NIFTY50,13.035213,20240502,22150.0,1,22144.95,0.131166,548.920601,0.009200,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=692768220, symbo...",150312.22,20.0
2,695293931,NIFTY50,20.035213,20240509,22150.0,1,22144.95,0.131166,680.530269,0.007421,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=695293931, symbo...",148280.97,20.0
3,696186822,NIFTY50,27.035213,20240516,22150.0,1,22144.95,0.131166,790.524553,0.006388,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=696186822, symbo...",146989.09,20.0
4,698202823,NIFTY50,34.035213,20240523,22150.0,1,22144.95,0.131166,886.981787,0.005693,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=698202823, symbo...",NaN,NaN
5,687901540,NIFTY50,41.035213,20240530,22150.0,1,22144.95,0.131166,973.932396,0.005185,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=687901540, symbo...",145661.59,20.0
6,694229717,NIFTY50,69.035213,20240627,22150.0,1,22144.95,0.131166,1263.239802,0.003998,C,NSE,50.0,[22150.0],"Contract(secType='OPT', conId=694229717, symbo...",145237.84,20.0


In [10]:
from utils import get_unds_with_prices
get_unds_with_prices(MARKET)

Qualifying NSE Unds:   0%|          | 0/179 [00:00<?, ?it/s]Error 200, reqId 43432: No security definition has been found for the request, contract: Stock(symbol='MIDCPNIFT', exchange='NSE', currency='INR')
Unknown contract: Stock(symbol='MIDCPNIFT', exchange='NSE', currency='INR')
Mkt prices with IVs: 100%|██████████| 5/5 [00:35<00:00,  7.16s/it]


,secType,conId,symbol,price,iv
0,IND,56994300,BANKNIFTY,47421.50,0.156918
1,IND,51497778,NIFTY50,22121.55,0.130965
2,STK,44652129,AMBUJACEM,611.15,0.320177
3,STK,166947150,ASTRAL,1942.60,0.286062
4,STK,269577540,BEL,233.05,0.365450
...,...,...,...,...,...
124,STK,56987148,RAMCOCEM,800.10,0.269863
125,STK,44652015,TCS,3840.55,0.213945
126,STK,217227809,ALKEM,4605.60,0.299605
127,STK,360973395,POLYCAB,5280.05,0.386013


In [ ]:
# make raw target options and qualify them
from utils import raw_target_opts
df_raw_opts = raw_target_opts(df_chains, 1)


In [ ]:
df_raw_opts

In [ ]:
df_opts = qualify_naked_opts(df_raw_opts)

In [ ]:
df_opts

In [ ]:
# get expected price for the target options
from utils import nakeds_with_expPrice
df_n = nakeds_with_expPrice(df_opts)

In [ ]:
df_n.head(3)

In [ ]:
3.45*50/99697.63*365/11.344245